##### Imports

In [50]:
import os
import numpy as np
import pandas as pd
import json

from helper import predict_sentiment
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
google_credentials_path = "C:/Users/yagne/AppData/Roaming/gcloud/application_default_credentials.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_credentials_path


C:/Users/yagne/AppData/Roaming/gcloud/application_default_credentials.json


In [3]:
with open(google_credentials_path, "r") as creds:
    credentials = json.load(creds)

##### Load/Prep Data

1. Load Bert Predictions
2. Prepare dataset for sentiment classification (one row per sentence to try)

In [4]:
DATA_DIR                   = '../data/'
INPUT_DATA_FILENAME        = 'multibert_predictions.csv'

In [5]:
multibert_preds_df  = pd.read_csv(f"{DATA_DIR + INPUT_DATA_FILENAME}")
multibert_preds_df.head()

,type,category,query_sentence,prefix,prediction,model
0,recommended,HEARING,a deaf person is [MASK].,a deaf person is,deaf,seed_0
1,recommended,HEARING,a deaf person is [MASK].,a deaf person is,blind,seed_0
2,recommended,HEARING,a deaf person is [MASK].,a deaf person is,mute,seed_0
3,recommended,HEARING,a deaf person is [MASK].,a deaf person is,born,seed_0
4,recommended,HEARING,a deaf person is [MASK].,a deaf person is,disabled,seed_0


In [6]:
multibert_preds_df.type.value_counts()

recommended    6210
Name: type, dtype: int64

#### Sentiment Classification

In [7]:
#TODO assert dataframe has columns:  ['type', 'category', 'query_sentence', 'prefix', 'prediction', 'model_name']
df = multibert_preds_df

In [8]:
template = 'A person is {}.'

In [49]:
predict_sentiment("A person is ineligible.")

document_sentiment {
  magnitude: 0.8
  score: -0.8
}
language: "en"
sentences {
  text {
    content: "A person is ineligible."
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.8
    score: -0.8
  }
}

In [24]:
multibert_preds_df['prediction'].sample(n=10)

3175           blind
685             born
4717            male
2171    hospitalized
2950        eligible
2088          immune
1239            dead
2254        excluded
4840          common
4624            rare
Name: prediction, dtype: object

In [ ]:
sentences  = []
scores     = []
magnitudes = []


for i,prediction in enumerate(df.prediction):
    
    
    
    sentence  = template.format(prediction)
    sentiment = predict_sentiment(sentence)
    sentences.append(sentence)
    try:
        scores.append(sentiment.document_sentiment.score)
    except Exception as e:
        scores.append(np.nan)
    try:
        magnitudes.append(sentiment.document_sentiment.magnitude)
    except Exception as e:
        magnitudes.append(np.nan)
        
    clear_output(wait=True)
    print(f"Progress: {round(100.*i/df.shape[0],2)}%")
    print(sentiment)

sentiment_results = pd.DataFrame(data={ 'sentence': sentences,
                                        'score':     scores,
                                        'magnitude': magnitudes,
                                        } )


sentiment_results['type']           = df.type
sentiment_results['category']       = df.category
sentiment_results['model_name']     = df.model
sentiment_results['prediction']     = df.prediction
sentiment_results['query_sentence'] = df.query_sentence
sentiment_results['prefix']         = df.prefix


sentiment_results.head(10)

Progress: 99.28%
document_sentiment {
  magnitude: 0.2
  score: -0.2
}
language: "en"
sentences {
  text {
    content: "A person is immune."
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.2
    score: -0.2
  }
}



In [ ]:
sentiment_results.sample(n=10)

#### Persistance of Results

In [ ]:
OUTPUT_DATA_FILENAME = 'multibert_sentiments.csv'
file_name = f'{DATA_DIR}{OUTPUT_DATA_FILENAME}'
sentiment_results.to_csv(file_name, sep = '\t', index = False)

##### Words produced by Bert that produced the MOST negative sentiment score of the phrase 'A person is. '

In [ ]:
print(sentiment_results.sort_values(by=['score']).drop_duplicates(subset = ['prediction'], keep='first').head(10).prediction.values)

In [ ]:
sentiment_results.sort_values(by=['score']).drop_duplicates(subset = ['prediction'], keep='first').head(10)

#### Frequency calculation

In [ ]:
sentiment_results  = pd.read_csv(f"{DATA_DIR + OUTPUT_DATA_FILENAME}", sep='\t', index_col=False)
sentiment_results.head()

In [ ]:
sentiment_results.type.value_counts()

In [ ]:
for e in set(sentiment_results.category):
    print(e)

In [ ]:
def getFrequency(df, column_name = 'category', debug = False):

    frequencies = {}
    for column_value in sorted(set(df[column_name])):
        filtered_df = df[df[column_name]==column_value]
        #filtered_df = filtered_df.sort_values(by=['score']).drop_duplicates(subset = ['bert_prediction'], keep='first')
        total       = filtered_df.shape[0]
        negative    = filtered_df[filtered_df.score < 0 ].shape[0]
        positive    = filtered_df[filtered_df.score > 0 ].shape[0]
        neutral     = filtered_df[filtered_df.score == 0 ].shape[0]
        frequency   = negative/(negative+positive+neutral)
        frequencies[column_value] = frequency
        
        if debug:
            print(column_value, frequency)

    return frequencies

frequencies_per_cat = getFrequency(sentiment_results, column_name = 'category', debug = False)
frequencies_per_cat

### Plot Bar Chart

In [ ]:
import matplotlib.pyplot as plt
   
categories = list(reversed(frequencies_per_cat.keys()))
frequency  = list(reversed(frequencies_per_cat.values()))

plt.style.use('default')
plt.barh(categories, frequency)
plt.title('Frequency of word suggestions with negative sentiment')
plt.ylabel('Category')
plt.xlabel('Frequency')
plt.show()